In [1]:
#!git clone https://github.com/remaplab/RecSys_Course_AT_PoliMi.git

Cloning into 'RecSys_Course_AT_PoliMi'...


remote: Enumerating objects: 1672, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (35/35), done.


remote: Total 1672 (delta 221), reused 209 (delta 209), pack-reused 1428 (from 3)
Receiving objects: 100% (1672/1672), 53.49 MiB | 27.11 MiB/s, done.


Resolving deltas: 100% (972/972), done.


In [2]:
#cd /kaggle/working/RecSys_Course_AT_PoliMi/

/kaggle/working/RecSys_Course_AT_PoliMi


In [3]:
#!git clone https://huggingface.co/datasets/reczoo/Movielens1M_m1

In [4]:
import json
import pandas as pd
import scipy as sp
import numpy as np
np.int = np.int32
np.float = np.float64
np.bool = np.bool_

In [6]:
from src.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from src.Evaluation.Evaluator import EvaluatorHoldout


urm_test=sp.sparse.load_npz("/splitted_urms/urm_test.npz")
urm_train=sp.sparse.load_npz("/splitted_urms/urm_train.npz")
urm_validation=sp.sparse.load_npz("/splitted_urms/urm_validation.npz")

evaluator = EvaluatorHoldout(urm_validation, cutoff_list=[10])


In [ ]:
from fairness_serendipity_fns import compute_fairness, compute_serendipity, compute_recommendations_dict,compute_recommendations_dict,compute_item_popularity
import json

with open('config_utils.json','r') as f:
    data=json.load(f)
    configurations_dict=data['weights_config']
    num_configs=data['num_configs']

In [8]:
item_popularity=compute_item_popularity(urm_train)
#α · Lhit rate + β1 · diversity + β2 · novelty + β3 · serendipity + β4 · fairness
def objective_function_slim(trial):
    params = {
        'l1_ratio': trial.suggest_float('l1_ratio', 1e-5, 1e-4, log=True),
        'alpha': trial.suggest_float('alpha', 1e-5, 0.5, log=True),
        'topK': trial.suggest_int('topK', 5, 200, log=True)
    }

    recommender_instance = SLIMElasticNetRecommender(urm_train)
    recommender_instance.fit(**params,positive_only=False)

    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    return result_df.loc[10, "HIT_RATE"]

def multi_objective_function_slim(trial):
    params = {
        'l1_ratio': trial.suggest_float('l1_ratio', 1e-5, 1e-4, log=True),
        'alpha': trial.suggest_float('alpha', 1e-5, 0.5, log=True),
        'topK': trial.suggest_int('topK', 5, 200, log=True)
    }

    recommender_instance = SLIMElasticNetRecommender(urm_train)
    recommender_instance.fit(**params,positive_only=False)
    rec_dict=compute_recommendations_dict(recommender_instance, urm_validation)
    
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    fairness=compute_fairness(rec_dict, item_popularity, G=4)
    serendipity=compute_serendipity(rec_dict, item_popularity)
    composed_metrics=weights_list[0]*result_df.loc[10, "HIT_RATE"] +  weights_list[1]*result_df.loc[10,"DIVERSITY_GINI"] +weights_list[2]*result_df.loc[10, "NOVELTY"]+ weights_list[3]*serendipity+weights_list[4]*fairness
    return composed_metrics

In [9]:
import optuna
from optuna.samplers import TPESampler

for i in range(1,num_configs+1):
    print(f"running config :{i}")
    config=f"c{i}"
    weights_list=configurations_dict[config]
    database_path=f"/kaggle/working/slim_multi_{config}.db"
    sampler=TPESampler(n_startup_trials=6)
    study=optuna.create_study(study_name=f"slim_multi_hyper_param_tuning_{config}",sampler=sampler,direction="maximize",storage=f'sqlite:///{database_path}',load_if_exists=True)
    study.optimize(multi_objective_function_slim, n_trials=50)


running config :4


[I 2025-11-16 13:44:30,003] A new study created in RDB with name: slim_multi_hyper_param_tuning_c4


SLIMElasticNetRecommender: Processed 995 (32.7%) in 5.00 min. Items per second: 3.31


SLIMElasticNetRecommender: Processed 2001 (65.8%) in 10.01 min. Items per second: 3.33


SLIMElasticNetRecommender: Processed 3001 (98.6%) in 15.01 min. Items per second: 3.33


...

SLIMElasticNetRecommender: Processed 2945 (96.8%) in 15.01 min. Items per second: 3.27


SLIMElasticNetRecommender: Processed 3043 (100.0%) in 15.50 min. Items per second: 3.27


EvaluatorHoldout: Processed 6022 (100.0%) in 2.83 sec. Users per second: 2127


[I 2025-11-17 01:35:53,411] Trial 48 finished with value: 0.33750967641566104 and parameters: {'l1_ratio': 1.1162120430643591e-05, 'alpha': 0.0001004027662412289, 'topK': 172}. Best is trial 47 with value: 0.44793506324497645.


SLIMElasticNetRecommender: Processed 956 (31.4%) in 5.00 min. Items per second: 3.18
